In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from kneed import KneeLocator
from geopy import distance  # geopy can only compute distances at surface
from sklearn.cluster import AgglomerativeClustering

In [3]:
def compute_elbow_threshold(df_label_counts, y_name="count", y_label="Number of samples"):
    """ Given an elbow curve, this functino finds the turning point using the KneeLocator. """
    x = df_label_counts.index
    y = df_label_counts[y_name]
    kn = KneeLocator(x=x, y=y, curve='convex', direction='increasing')
    knee = kn.knee

    fig = plt.figure()
    plt.xlabel('Clusters')
    plt.ylabel(y_label)
    plt.plot(x, y, "-")  # "'x-')
    plt.vlines(knee, plt.ylim()[0], plt.ylim()[1], linestyles='dashed', color="orange")
    plt.hlines(df_label_counts.iloc[knee][y_name], plt.xlim()[0], plt.xlim()[1], linestyles='dashed', color="orange")
    # plt.xticks([knee], labels=[df_label_counts.iloc[knee]["labels"]])
    plt.xticks([knee])
    plt.yticks(list(plt.yticks()[0]) + [df_label_counts.iloc[knee][y_name]])
    plt.show()

    return df_label_counts.iloc[knee]["labels"], df_label_counts.iloc[knee][y_name]

In [4]:
def coupled_label_plot(df):
    """ Plots the clusters in geographical and UMAP space in matching colours. """
    map = Basemap(llcrnrlon=df["LONGITUDE"].min(),
                  llcrnrlat=df["LATITUDE"].min(),
                  urcrnrlon=df["LONGITUDE"].max(),
                  urcrnrlat=df["LATITUDE"].max())

    # plot clusters in 3D
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')
    ax.scatter(df["LONGITUDE"], df["LATITUDE"], df["LEV_M"], color=df["color"], s=2, alpha=1, zorder=4)
    ax.add_collection3d(map.drawcoastlines(linewidth=0.5))
    ax.set_box_aspect((np.ptp(df["LONGITUDE"]), np.ptp(df["LATITUDE"]),
                       np.ptp(df["LEV_M"]) / 50))  # aspect ratio is 1:1:1 in data space
    plt.gca().invert_zaxis()
    plt.show()

    # plot clusters in UMAP space
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')
    ax.scatter(df["e0"], df["e1"], df["e2"], c=df["color"], alpha=0.8, zorder=4, s=1)  # , s=s, alpha=1, zorder=4)
    plt.show()

In [5]:
def compute_volume(row, dlat, dlon, depths):
    # careful!! I am assuming that depth does not affect distance calculations
    # approximate volume as a box, ignore depth, surface approximated by trapezoid
    lat = row["LATITUDE"]
    lon = row["LONGITUDE"]
    depth = row["LEV_M"]

    # find depth step size
    idx = np.argwhere(depths == depth)
    next_depth = depths[idx+1]
    ddepth = (next_depth - depth).flatten()[0]

    # compute box edge points
    p0 = np.array([lat - dlat / 2, lon - dlon / 2, depth])
    p1 = np.array([lat - dlat / 2, lon + dlon / 2, depth])
    p2 = np.array([lat + dlat / 2, lon + dlon / 2, depth])
    p3 = np.array([lat + dlat / 2, lon - dlon / 2, depth])

    # compute distances
    # d03 = distance.distance(p0, p3).m
    d01 = distance.distance(p0, p1).m
    d23 = distance.distance(p2, p3).m
    # d12 = distance.distance(p2, p1).m

    # compute height of trapezoid
    p01 = np.array([p0[0], lon, p0[2]])
    p23 = np.array([p2[0], lon, p2[2]])
    h = distance.distance(p01, p23).m

    # compute volume
    # v = round((depth+ ddepth) * 0.5 * (d01 + d23) * h, 4)  # volume in m3
    v = ddepth * 0.5 * (d01 + d23) * h  # volume in m3

    return v

In [6]:
def drop_clusters_with_few_samples(df, thresh=None):
    """ If thresh is None, the Kneedle algorithm will be used to determine a treshold. """
    # count number of grid cells in each cluster
    df_nums = df.groupby("labels").count()["color"].reset_index().rename(columns={"color": "count"})
    df_nums = df_nums.sort_values("count").reset_index(drop=True)
    df_nums["labels"] = df_nums["labels"].astype(str)

    # compute threshold to cut off clusters
    if not thresh:
        cluster_label, thresh = compute_elbow_threshold(df_nums, y_name="count", y_label="Number of samples")
    else:
        # plot number of cells per cluster
        plt.plot(df_nums["labels"], df_nums["count"])
        # plt.yscale("log")
        plt.ylabel("Number of samples")
        plt.xlabel("Cluster")
        plt.xticks([])
        plt.axhline(thresh, color="orange")
        plt.yticks(list(plt.yticks()[0]) + [thresh])
        plt.show()

    # set all labels to -1 (noise) where num samples is too small
    labels_to_keep = df_nums[df_nums["count"] >= thresh].labels
    print("Remaining number of clusters:" + str(len(labels_to_keep)))
    df_new = df[df["labels"].isin(labels_to_keep)]

    # plotting
    coupled_label_plot(df_new)

    return df_new

In [7]:
def drop_clusters_with_small_volume(df, thresh=None):
    # get depth resolution
    depths = df["LEV_M"].unique()
    depths = np.concatenate([depths, np.array([5000]).reshape(1, -1).flatten()])  # adding the lower depth bound

    # compute volume
    df["volume"] = df.apply(compute_volume, axis=1, args=(dlat, dlon, depths))  # careful with rounding

    # check cluster volumes
    df_vols = df.groupby("labels").sum()["volume"].reset_index()
    df_vols = df_vols.sort_values("volume").reset_index(drop=True)
    df_vols["labels"] = df_vols["labels"].astype(str)

    if not thresh:
        cluster_label, thresh = compute_elbow_threshold(df_vols, y_name="volume", y_label="Cluster volume")
    else:
        # plot cluster volumes
        plt.plot(df_vols["labels"], df_vols["volume"])
        plt.xlabel("Cluster")
        plt.xticks([])
        plt.ylabel("Cluster volume [$m^3$]")
        # plt.yscale("log")
        # plt.ylabel(r"log(volume) [$m^3$]")
        plt.yticks(list(plt.yticks()[0]) + [thresh])
        plt.axhline(thresh, color="orange")
        plt.show()

    # decide which labels to keep
    labels_to_keep = df_vols[df_vols["volume"] >= thresh].labels
    print("Remaining number of clusters:" + str(len(labels_to_keep)))
    df_new = df[df["labels"].isin(labels_to_keep)]

    # plot new data in 3D and UMAP space
    coupled_label_plot(df_new)

    return df_new


In [8]:
# load data
df = pd.read_csv("data/dbscan_on_embedding.csv")
df["labels"] = df["labels"].astype(str)

FileNotFoundError: [Errno 2] No such file or directory: 'data/dbscan_on_embedding.csv'

In [ ]:
# plot DBSCAN labels in 3D and UMAP space
coupled_label_plot(df)

In [ ]:
# drop clusters with few samples (manual thresh: 60)
df_s = drop_clusters_with_few_samples(df, None)

In [ ]:
# save df
# df_s.to_csv("output/dbscan_on_embedding_sampleDropped.csv", index=False)

In [ ]:
# drop clusters with small volume (manual thresh: 3e14)
df_v = drop_clusters_with_small_volume(df, None)

In [ ]:
# save df
# df_v.to_csv("output/dbscan_on_embedding_volumeDropped.csv", index=False)